<h1><center>Fanlore.org scraping project<center>
<center>DSBA course<center>
<center>Organised by Karol Rochalski<center><h1>

In [2]:
from bs4 import BeautifulSoup
import pathlib
import os
import pandas as pd
import numpy as np
import regex as re
import requests
from selenium import webdriver
import shutil

Url of website we are going to scrape is stored in external .txt file. This is conducted for safety reasons.
We access the file and read the first line, then assign it to the variable.

In [3]:

file = open('./urls.txt', 'r')
url = file.readline()
url

'https://fanlore.org/wiki/Category:Fan_Communities'

In [4]:
driver = webdriver.Chrome()
driver.quit()

Default request to crawl the website. 
It takes url from .txt file and operates on them in order to find certain data.
Below that new variable 'soup' has standard BeautifulSoup parser.

In [5]:

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')


Outputs content of the website. This is not desired format. In fact it's barely readable.
The big advantage of it is that we get a good grasp of what we are going to work with and 
confirm the website is responsive. It selves as a good starting point.

In [6]:
print(soup.title)
print(soup.prettify())

<title>Category:Fan Communities - Fanlore</title>
<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Category:Fan Communities - Fanlore
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"690c8cf06be0ce7bbef03513","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Fan_Communities","wgTitle":"Fan Communities","wgCurRevisionId":2508947,"wgRevisionId":2508947,"wgArticleId":382,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Fanlore Categories"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRelev

<h2>Testing basics</h2>

The main format of the project consists of groups of scraped values put into numpy arrays. 
The project's final output will summarize dataframes utilizing main sections of the website: Subcategories and Fan Communities with regard to their information (names, links, sections). The other data will provide scraped images and other minor data.

In [7]:
div = soup.find_all('div', class_='')
divs = []
for x in div:
    divs.append(str(x))
#divs.append(str(div))
divsNP = np.array(divs)

In [8]:
#Details of the website

#head data
haed = soup.find('head')


#footer data
footer = soup.find('footer')

<h2>Subcategories</h2>

In this part we are extracting Subcategory information. The main goal is to get titles and external links.
We extract the formula from the div, proceed to the span tag and then extract titles from data-ct-title attribute.

In [9]:

#Subcategories
Subcategory =  soup.find('div', id ='mw-subcategories').findAll('a')

Subcategories = []
for x in Subcategory:
    attr = x.attrs

    n = attr.get('title')
    #slicing beginning of the string
    n = n[9:]

    Subcategories.append(n)

SubcategoriesNPTitles = np.array(Subcategories)
print(SubcategoriesNPTitles[0])
print(type(SubcategoriesNPTitles[0]))


Communities by Fandom
<class 'numpy.str_'>


For links we are gonna copy-paste same code and modify it to extract the links.

In [10]:
Subcategory =  soup.find('div', id ='mw-subcategories').findAll('a')

Subcategories = []
for x in Subcategory:
    attr = x.attrs
    #print(attr)
    Subcategories.append(attr.get('href'))

SubcategoriesNPLinks = np.array(Subcategories)
print(SubcategoriesNPLinks.size)
for x in SubcategoriesNPLinks:
    print(x)
print(type(SubcategoriesNPLinks[1]))

17
/wiki/Category:Communities_by_Fandom
/wiki/Category:Communities_by_Year
/wiki/Category:Discord_Servers
/wiki/Category:Fandoms_by_Source_Text
/wiki/Category:Fandoms_by_Canon_Type
/wiki/Category:Fandoms_by_Community
/wiki/Category:Forums
/wiki/Category:Imageboards
/wiki/Category:Journaling_Communities
/wiki/Category:Mailing_Lists
/wiki/Category:Microblogs
/wiki/Category:Multimedia_Communities
/wiki/Category:Newsgroups
/wiki/Category:Offline_Spaces
/wiki/Category:Social_Networking
/wiki/Category:Subreddits
/wiki/Category:Websites
<class 'numpy.str_'>


Now we can combine columns into single dataframe.

In [11]:
column_name = ['Titles', 'Links', 'Nr. of child categories', 'Nr. of child Pages', 'Nr. of child files']
df = pd.DataFrame({column_name[0]: SubcategoriesNPTitles, column_name[1]: SubcategoriesNPLinks})
df

,Titles,Links
0,Communities by Fandom,/wiki/Category:Communities_by_Fandom
1,Communities by Year,/wiki/Category:Communities_by_Year
2,Discord Servers,/wiki/Category:Discord_Servers
3,Fandoms by Source Text,/wiki/Category:Fandoms_by_Source_Text
4,Fandoms by Canon Type,/wiki/Category:Fandoms_by_Canon_Type
5,Fandoms by Community,/wiki/Category:Fandoms_by_Community
6,Forums,/wiki/Category:Forums
7,Imageboards,/wiki/Category:Imageboards
8,Journaling Communities,/wiki/Category:Journaling_Communities
9,Mailing Lists,/wiki/Category:Mailing_Lists


In [12]:
#Fan Communities listed
soup.find_all('li', 'a', 'title')

[]

<h2> Image extraction </h2>

Now it's time for the funniest part - image extraction. Even though we don't work with fan subscription data, there's still a lot to take in consideration.

In [15]:
#attached pictures and/or audio
images = soup.find_all('img') 

#make a folder for images

#example = images[0]

for i, x in enumerate(images):
    image_link = x.attrs['src']
    print(image_link)
    full_url = url + image_link
    r = requests.get(full_url).content
    r = str(r, 'utf-8')
    with open(f"./images/image{i+1}.jpg", "wb+") as f:
        f.write(r)

#This approach doesn't work as the structure of the website is a bit different

filename = re.search(r'/([\w_-]+[.](jpg|gif|png))$', url)
    

/w/images/thumb/2/20/Christianfanfellowship.jpg/138px-Christianfanfellowship.jpg 1.5x, /w/images/thumb/2/20/Christianfanfellowship.jpg/184px-Christianfanfellowship.jpg 2x


KeyboardInterrupt: 

In [21]:
#Internal links leading outside of the website
media = soup.find('div', {"id" :"mw-category-media"})
links = soup.find_all('a')
for link in links:
    print("Link: ", link.get('href'), 'Text: ', link.string)

Link:  None Text:  None
Link:  https://www.mediawiki.org/wiki/Special:MyLanguage/Help:Categories Text:  Help
Link:  #mw-head Text:  Jump to navigation
Link:  #searchInput Text:  Jump to search
Link:  /wiki/Community Text:  communities
Link:  /wiki/Fandom Text:  fandoms
Link:  /wiki/Category:Fandoms_by_Source_Text Text:  Fandoms by Source Text
Link:  /wiki/Category:Fandoms_by_Community Text:  Fandoms by Community
Link:  /wiki/Category:Fandoms_by_Canon_Type Text:  Fandoms by Canon Type
Link:  /wiki/Fandom Text:  fandom
Link:  /wiki/Category:Fandoms_by_Source_Text Text:  Fandoms by Source Text
Link:  /wiki/Fandoms Text:  fandoms
Link:  /wiki/Supernatural Text:  Supernatural
Link:  /wiki/Category:Fandoms_by_Community Text:  Fandoms by Community
Link:  /wiki/Media_Fandom Text:  Media Fandom
Link:  /wiki/Vidding_Fandom Text:  Vidding Fandom
Link:  /wiki/Category:Fandoms_by_Canon_Type Text:  Fandoms by Canon Type
Link:  /wiki/Category:Offline_Spaces Text:  Offline Spaces
Link:  /wiki/Fan_Club